In [1]:
import os
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11.0.11"
os.environ["SPARK_HOME"] = f"C:\spark\spark-3.1.2-bin-hadoop3.2"
import findspark
import pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("emissionsdataframe").getOrCreate()
from sqlalchemy import create_engine, insert
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData, update, Table
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import config
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col
from pyspark.sql import Row


In [2]:
rds_string = config.rds_string
engine = create_engine(f'postgresql://{rds_string}')
conn = engine.connect()
metadata = MetaData(engine)

In [3]:
def sql_to_spark(table):
    from pyspark.sql import Row
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).all()
    steve = table_var.metadata.tables[table].columns.keys()
    query_list = []
    for i in query:
        q_len = len(i)
        temp_dict = {}
        for j in range(q_len):
            key = steve[j]
            value = i[j]
            if value == None:
                value = float(0)
            temp_dict[key] = value
            if j == (q_len - 1):
                query_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in query_list)
    return df

def query_maker(table):
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).all()
    steve = table_var.metadata.tables[table].columns.keys()
    return (query, steve)

def spark_maker(query,country):
    c_list = []
    for i in query:
        q_len = len(i)
        if i[0] == country:
            temp_dict = {}
            for j in range(q_len):
                key = steve[j]
                value = i[j]
                if value == None:
                    value = float(0)
                temp_dict[key] = value
                if j == (q_len - 1):
                    c_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in c_list) 
    return df

In [ ]:
# metadata = MetaData(engine)

# global_trade = Table("global_trade", metadata, autoload=True, autoload_with=engine)
# Session = sessionmaker()
# Session.configure(bind=engine)
# session = Session()
# query = session.query(global_trade).all()

In [ ]:
sparkDF.filter(sparkDF["Country "] == "Afghanistan").select(['Country ']).show()

In [ ]:
sparkDF.select(['Country ']).show()

In [4]:
spark_gdp=sql_to_spark('gdp_data')

In [5]:
spark_india=sql_to_spark("india_export_data")

In [6]:
spark_emissions=sql_to_spark('global_emissions')

In [36]:
germany_spark = spark_maker(query, "Germany")

In [61]:
new_germany_spark = germany_spark.withColumn("year",germany_spark.year.cast('int'))

In [63]:
new_germany_spark.printSchema()

root
 |-- country_or_area: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- comm_code: string (nullable = true)
 |-- commodity: string (nullable = true)
 |-- flow: string (nullable = true)
 |-- trade_usd: long (nullable = true)
 |-- weight_kg: double (nullable = true)
 |-- quantity_name: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- category: string (nullable = true)



In [ ]:
df.orderBy(desc("age"), "name").collect()

In [69]:
new_germany_spark.orderBy(new_germany_spark.year.asc()).show(1)

+---------------+----+---------+----------+------+---------+-------------+-------------------+-------------+--------------------+
|country_or_area|year|comm_code| commodity|  flow|trade_usd|    weight_kg|      quantity_name|     quantity|            category|
+---------------+----+---------+----------+------+---------+-------------+-------------------+-------------+--------------------+
|        Germany|1991|   120100|Soya beans|Import|695756992|2.878631168E9|Weight in kilograms|2.878631168E9|12_oil_seed_oleag...|
+---------------+----+---------+----------+------+---------+-------------+-------------------+-------------+--------------------+
only showing top 1 row

